In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

E0000 00:00:1730823167.729307   41610 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730823167.814882   41610 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import matplotlib.pyplot as plt

def plot_learning_curves(history):
    """
    Plot training and validation accuracy and loss curves from the model's history.

    Parameters:
    history (History): The History object returned by model.fit, which contains
                       training and validation accuracy and loss per epoch.
    """
    # Get training and validation accuracy values
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    # Get training and validation loss values
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Plot accuracy
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [4]:
from tqdm import tqdm
import cv2 as cv
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

hand_signs = os.listdir("data/train")

images = []
labels = []

for sign in hand_signs:
    path = "data/train/" + sign

    for image in tqdm(os.listdir(path), desc=f"Loading images for {sign}"):
        img = cv.imread(path + "/" + image)
        img = cv.resize(img, (640, 380))
        if img is not None:
            images.append(img)
            labels.append(sign)

print("\nConverting training images to np array with float32 values scaled")
X = np.array(images).astype("float32") / 255.

print("\nEncoding Y properly to a understandable format")
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

print("\nOne Hot Encoding training labels")
y = y.reshape(-1, 1)
one_hot_encoder = OneHotEncoder()
Y = one_hot_encoder.fit_transform(y)

Loading images for dog:  68%|██████▊   | 179/263 [00:03<00:01, 58.31it/s]


KeyboardInterrupt: 

In [ ]:
hand_signs = os.listdir("data/test")

images_test = []
labels_test = []

for sign in hand_signs:
    path = "data/test/" + sign

    for image in tqdm(os.listdir(path), desc=f"Loading images for {sign}"):
        img = cv.imread(path + "/" + image)
        img = cv.resize(img, (640, 380))
        if img is not None:
            images_test.append(img)
            labels_test.append(sign)

print("\nConverting training images to np array with float32 values scaled")
X_test = np.array(images_test).astype("float32") / 255.0

print("\nEncoding Y properly to a understandable format")
label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(labels_test)

print("\nOne Hot Encoding training labels")
y_test = y_test.reshape(-1, 1)
one_hot_encoder = OneHotEncoder()
Y_test = one_hot_encoder.fit_transform(y_test)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

def create_vgg_model(input_shape=X[0].shape, num_classes=13):
    with tf.device("/gpu:0"):
        vgg_model = VGG16(
            include_top=False,
            input_shape=input_shape,
            pooling="avg",
            weights="imagenet"
        )

        for layer in vgg_model.layers[:-5]:
            layer.trainable = False

        x = Dense(4096, activation="relu")(vgg_model.output)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        x = Dense(1024, activation="relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        output = Dense(num_classes, activation="softmax", name="Prediction")(x)

        model = Model(inputs=vgg_model.input, outputs=output, name="Sharingan")

    return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.sparse import issparse

def train_model(model, X, Y, X_test, Y_test):
    with tf.device("/gpu:0"):
        if issparse(X):
            X = X.toarray()
        if issparse(Y):
            Y = Y.toarray()
        if issparse(X_test):
            X_test = X_test.toarray()
        if issparse(Y_test):
            Y_test = Y_test.toarray()

        X = np.asarray(X)
        Y = np.asarray(Y)
        X_test = np.asarray(X_test)
        Y_test = np.asarray(Y_test)

        train_datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            brightness_range=[0.8, 1.2],
            preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
        )

        val_datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
        )

        model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=['accuracy']
        )

        batch_size = 48
        val_size = 32

        callbacks = [
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_accuracy',
                factor=0.5,
                patience=3,
                min_delta=0.001,
                min_lr=1e-9
            ),
            tf.keras.callbacks.ModelCheckpoint(
                './models/best_VGG_Classifier.keras',
                save_best_only=True,
                monitor='val_accuracy'
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                min_delta=0.001,
                mode='min'
            )
        ]

        train_gen = train_datagen.flow(X, Y, batch_size=batch_size)
        val_gen = val_datagen.flow(X_test, Y_test, batch_size=val_size)

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            steps_per_epoch=len(X) // batch_size,
            validation_steps=None,
            epochs=100,
            callbacks=callbacks,
        )

        return history

In [ ]:
model = create_vgg_model(num_classes=len(hand_signs))
model.summary()
history = train_model(model, X, Y, X_test, Y_test)

In [ ]:
plot_learning_curves(history)

In [ ]:
model.save("./models/final_VGG_Classifier.keras")